In [1]:
import tensorflow as tf
#config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth=True
#sess = tf.compat.v1.Session(config=config)
tf.compat.v1.enable_eager_execution()

In [2]:
from tensorflow import keras
import tensorflow.keras.backend as K
#K.clear_session()
from tensorflow.keras import layers

In [3]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [4]:
import numpy as np
from sklearn.model_selection import LeaveOneOut,KFold
from sklearn.datasets import load_iris

In [5]:
import efrc_ml_production as ml

/home/modules/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/modules/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [6]:
n_splits = 10
data = load_iris()
X=data['data']
y=data['target']

def tf_make_dataset(X_data,y_data,n_splits):

    def gen():
        for train_index, test_index in KFold(n_splits).split(X_data):
            X_train, X_test = X_data[train_index], X_data[test_index]
            y_train, y_test = y_data[train_index], y_data[test_index]
            yield X_train,y_train,X_test,y_test

    return tf.data.Dataset.from_generator(gen, (tf.float64,tf.float64,tf.float64,tf.float64))

dataset=tf_make_dataset(X,y,5)

In [7]:
BATCH_SIZE = 5

In [8]:
#dataset=dataset.cache().batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
def build_model(n_features, lr, h_units, ACTIVATION):
    #x = tf.placeholder('float', shape = [None, n_features])
    model = keras.Sequential([
        layers.Dense(h_units, activation='relu', input_shape=[n_features]), #default is 100
        #layers.Dense(100, activation='relu'), #default is not exist
        #layers.Dropout(.1), #default is not exist
        #layers.Dense(100, activation='relu'), #default is not exist
        #layers.Dropout(.1), #default is not exist
        layers.Dense(h_units, activation='relu'), #default is 100
        #layers.Dropout(.1), #default is not exist
        layers.Dense(h_units, activation='relu'), #default is 100
        #layers.Dropout(.1), #default is not exist
        #layers.Dense(1)
        layers.Dense(1, activation='linear') #default activation is None
    ])

#     model = keras.Sequential([
#         layers.Dense(400, activation='relu', input_shape=[len(train_fp.keys())]),
#         layers.Dense(400, activation='relu'),
#         layers.Dense(100, activation='relu'),
#         #layers.Dense(1)
#         layers.Dense(1, activation='linear')
#     ])

    opt = keras.optimizers.Adam(learning_rate=lr) #default is .001
    
    model.compile(loss='mse',
        optimizer=opt,
        metrics=['mae', 'mse'])
    return model

In [10]:
model = build_model(X.shape[1], .001, 20, 'relu')

In [11]:
import os

In [12]:
import time

In [13]:
#using (X_test, y_test) as validation
start = time.time()
import datetime
rmses = []
chkpt_name = 'model_checkpoint'
patience=10
os.system('rm %s.h5' %chkpt_name)
try:
    os.system('rm -rf logs')
except:
    pass
for X_train,y_train,X_test,y_test in dataset:
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    checkpoint_callbacks = keras.callbacks.ModelCheckpoint(filepath='%s.h5' %chkpt_name, monitor='val_loss',\
                                                          verbose=1, save_best_only=True, mode='min')

#     log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    model = build_model(X.shape[1], .001, 20, 'relu')

    model.fit(X_train, y_train, batch_size=5,epochs=25, verbose=1, callbacks=[checkpoint_callbacks, early_stop],
    #tfdocs.modeling.EpochDots(),tensorboard_callback],
    validation_split=None, validation_data=(X_test,y_test), shuffle=True, class_weight=None,
    sample_weight=None, initial_epoch=0, steps_per_epoch=None,
    validation_steps=None, validation_freq=1, max_queue_size=10, workers=1,
    use_multiprocessing=False) 
    model.load_weights(filepath='%s.h5' %chkpt_name)
    preds = model.predict(X_test)
    rmses.append(ml.get_rmse(preds, y_test))
print("Best Models Average RMSE: %s" %np.mean(rmses))
end = time.time()
print('Done in %s' %(end-start))

Train on 120 samples, validate on 30 samples
Epoch 1/25
115/120 [===========================>..] - ETA: 0s - loss: 2.7308 - mae: 1.4412 - mse: 2.7308 
Epoch 00001: val_loss improved from inf to 0.00081, saving model to model_checkpoint.h5
120/120 [==============================] - 1s 8ms/sample - loss: 2.6497 - mae: 1.4101 - mse: 2.6497 - val_loss: 8.0946e-04 - val_mae: 0.0238 - val_mse: 8.0946e-04
Epoch 2/25
  5/120 [>.............................] - ETA: 0s - loss: 0.6553 - mae: 0.5812 - mse: 0.6553
Epoch 00002: val_loss did not improve from 0.00081
120/120 [==============================] - 0s 584us/sample - loss: 0.2598 - mae: 0.3951 - mse: 0.2598 - val_loss: 0.0765 - val_mae: 0.2695 - val_mse: 0.0765
Epoch 3/25
  5/120 [>.............................] - ETA: 0s - loss: 0.0608 - mae: 0.2365 - mse: 0.0608
Epoch 00003: val_loss did not improve from 0.00081
120/120 [==============================] - 0s 607us/sample - loss: 0.1161 - mae: 0.3037 - mse: 0.1161 - val_loss: 0.0066 - val_ma

Epoch 8/25
115/120 [===========================>..] - ETA: 0s - loss: 0.0451 - mae: 0.1564 - mse: 0.0451
Epoch 00008: val_loss did not improve from 0.01214
120/120 [==============================] - 0s 689us/sample - loss: 0.0484 - mae: 0.1620 - mse: 0.0484 - val_loss: 0.0220 - val_mae: 0.1020 - val_mse: 0.0220
Epoch 9/25
115/120 [===========================>..] - ETA: 0s - loss: 0.0506 - mae: 0.1613 - mse: 0.0506
Epoch 00009: val_loss did not improve from 0.01214
120/120 [==============================] - 0s 698us/sample - loss: 0.0581 - mae: 0.1712 - mse: 0.0581 - val_loss: 0.0355 - val_mae: 0.1249 - val_mse: 0.0355
Epoch 10/25
  5/120 [>.............................] - ETA: 0s - loss: 0.0789 - mae: 0.1870 - mse: 0.0789
Epoch 00010: val_loss did not improve from 0.01214
120/120 [==============================] - 0s 679us/sample - loss: 0.0491 - mae: 0.1614 - mse: 0.0491 - val_loss: 0.0243 - val_mae: 0.1069 - val_mse: 0.0243
Epoch 11/25
  5/120 [>.............................] - ETA: 

Epoch 9/25
  5/120 [>.............................] - ETA: 0s - loss: 0.0076 - mae: 0.0524 - mse: 0.0076
Epoch 00009: val_loss did not improve from 0.05573
120/120 [==============================] - 0s 595us/sample - loss: 0.0331 - mae: 0.1203 - mse: 0.0331 - val_loss: 0.1345 - val_mae: 0.2955 - val_mse: 0.1345
Epoch 10/25
115/120 [===========================>..] - ETA: 0s - loss: 0.0274 - mae: 0.1129 - mse: 0.0274
Epoch 00010: val_loss did not improve from 0.05573
120/120 [==============================] - 0s 620us/sample - loss: 0.0268 - mae: 0.1116 - mse: 0.0268 - val_loss: 0.1354 - val_mae: 0.2957 - val_mse: 0.1354
Epoch 11/25
  5/120 [>.............................] - ETA: 0s - loss: 0.0577 - mae: 0.1623 - mse: 0.0577
Epoch 00011: val_loss did not improve from 0.05573
120/120 [==============================] - 0s 585us/sample - loss: 0.0272 - mae: 0.1138 - mse: 0.0272 - val_loss: 0.1009 - val_mae: 0.2452 - val_mse: 0.1009
Epoch 12/25
  5/120 [>.............................] - ETA:

Epoch 22/25
105/120 [=========================>....] - ETA: 0s - loss: 0.0493 - mae: 0.1546 - mse: 0.0493
Epoch 00022: val_loss did not improve from 0.05140
120/120 [==============================] - 0s 694us/sample - loss: 0.0479 - mae: 0.1524 - mse: 0.0479 - val_loss: 0.0514 - val_mae: 0.1891 - val_mse: 0.0514
Epoch 23/25
 95/120 [======================>.......] - ETA: 0s - loss: 0.0535 - mae: 0.1602 - mse: 0.0535
Epoch 00023: val_loss improved from 0.05140 to 0.05000, saving model to model_checkpoint.h5
120/120 [==============================] - 0s 1ms/sample - loss: 0.0478 - mae: 0.1521 - mse: 0.0478 - val_loss: 0.0500 - val_mae: 0.1931 - val_mse: 0.0500
Epoch 24/25
115/120 [===========================>..] - ETA: 0s - loss: 0.0448 - mae: 0.1455 - mse: 0.0448
Epoch 00024: val_loss did not improve from 0.05000
120/120 [==============================] - 0s 682us/sample - loss: 0.0432 - mae: 0.1419 - mse: 0.0432 - val_loss: 0.0602 - val_mae: 0.1932 - val_mse: 0.0602
Epoch 25/25
  5/120

Epoch 22/25
  5/120 [>.............................] - ETA: 0s - loss: 0.1006 - mae: 0.2672 - mse: 0.1006
Epoch 00022: val_loss did not improve from 0.07364
120/120 [==============================] - 0s 584us/sample - loss: 0.0327 - mae: 0.1217 - mse: 0.0327 - val_loss: 0.1391 - val_mae: 0.3303 - val_mse: 0.1391
Epoch 23/25
  5/120 [>.............................] - ETA: 0s - loss: 0.0105 - mae: 0.0658 - mse: 0.0105
Epoch 00023: val_loss did not improve from 0.07364
120/120 [==============================] - 0s 585us/sample - loss: 0.0311 - mae: 0.1139 - mse: 0.0311 - val_loss: 0.1316 - val_mae: 0.3187 - val_mse: 0.1316
Epoch 24/25
  5/120 [>.............................] - ETA: 0s - loss: 0.0192 - mae: 0.1089 - mse: 0.0192
Epoch 00024: val_loss did not improve from 0.07364
120/120 [==============================] - 0s 481us/sample - loss: 0.0349 - mae: 0.1231 - mse: 0.0349 - val_loss: 0.1236 - val_mae: 0.3057 - val_mse: 0.1236
Epoch 25/25
  5/120 [>.............................] - ETA

In [45]:
#with validation specification
rmses = []
chkpt_name = 'model_checkpoint_w_val'
os.system('rm %s.h5' %chkpt_name)
os.system('rm -rf logs')
for X_train,y_train,X_test,y_test in dataset:
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    checkpoint_callbacks = keras.callbacks.ModelCheckpoint(filepath='%s.h5' %chkpt_name, monitor='val_loss',\
                                                          verbose=1, save_best_only=True, mode='min')

    log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    model = build_model(X.shape[1], .001, 20, 'relu')
    
    model.fit(X_train, y_train, batch_size=5,epochs=100, verbose=1, callbacks=[checkpoint_callbacks,
    tfdocs.modeling.EpochDots(),tensorboard_callback],
    validation_split=.1, shuffle=True, class_weight=None,
    sample_weight=None, initial_epoch=0, steps_per_epoch=None,
    validation_steps=None, validation_freq=1, max_queue_size=10, workers=1,
    use_multiprocessing=False)
    model.load_weights(filepath='%s.h5' %chkpt_name)
    preds = model.predict(X_test)
    rmses.append(ml.get_rmse(preds, y_test))
print("Best Model RMSE: %s" %np.mean(rmses))

Train on 108 samples, validate on 12 samples
Epoch 1/100
100/108 [==========================>...] - ETA: 0s - loss: 0.1653 - mae: 0.3425 - mse: 0.1653
Epoch 00001: val_loss improved from inf to 0.11442, saving model to model_checkpoint_w_val.h5

Epoch: 0, loss:0.1558,  mae:0.3293,  mse:0.1558,  val_loss:0.1144,  val_mae:0.3117,  val_mse:0.1144,  
108/108 [==============================] - 0s 4ms/sample - loss: 0.1558 - mae: 0.3293 - mse: 0.1558 - val_loss: 0.1144 - val_mae: 0.3117 - val_mse: 0.1144
Epoch 2/100
100/108 [==========================>...] - ETA: 0s - loss: 0.0864 - mae: 0.2388 - mse: 0.0864
Epoch 00002: val_loss improved from 0.11442 to 0.11116, saving model to model_checkpoint_w_val.h5
108/108 [==============================] - 0s 901us/sample - loss: 0.0870 - mae: 0.2385 - mse: 0.0870 - val_loss: 0.1112 - val_mae: 0.2982 - val_mse: 0.1112
Epoch 3/100
100/108 [==========================>...] - ETA: 0s - loss: 0.0603 - mae: 0.1955 - mse: 0.0603
Epoch 00003: val_loss did not

Epoch 26/100
 85/108 [======================>.......] - ETA: 0s - loss: 0.0289 - mae: 0.1311 - mse: 0.0289
Epoch 00026: val_loss did not improve from 0.03436
108/108 [==============================] - 0s 829us/sample - loss: 0.0419 - mae: 0.1499 - mse: 0.0419 - val_loss: 0.0696 - val_mae: 0.2313 - val_mse: 0.0696
Epoch 27/100
 85/108 [======================>.......] - ETA: 0s - loss: 0.0382 - mae: 0.1393 - mse: 0.0382
Epoch 00027: val_loss did not improve from 0.03436
108/108 [==============================] - 0s 843us/sample - loss: 0.0408 - mae: 0.1459 - mse: 0.0408 - val_loss: 0.0745 - val_mae: 0.2311 - val_mse: 0.0745
Epoch 28/100
 85/108 [======================>.......] - ETA: 0s - loss: 0.0434 - mae: 0.1559 - mse: 0.0434
Epoch 00028: val_loss did not improve from 0.03436
108/108 [==============================] - 0s 837us/sample - loss: 0.0451 - mae: 0.1583 - mse: 0.0451 - val_loss: 0.0772 - val_mae: 0.2353 - val_mse: 0.0772
Epoch 29/100
 95/108 [=========================>....] -

Epoch 52/100
 85/108 [======================>.......] - ETA: 0s - loss: 0.0407 - mae: 0.1478 - mse: 0.0407
Epoch 00052: val_loss did not improve from 0.03098
108/108 [==============================] - 0s 824us/sample - loss: 0.0454 - mae: 0.1531 - mse: 0.0454 - val_loss: 0.0379 - val_mae: 0.1650 - val_mse: 0.0379
Epoch 53/100
 90/108 [========================>.....] - ETA: 0s - loss: 0.0345 - mae: 0.1317 - mse: 0.0345
Epoch 00053: val_loss did not improve from 0.03098
108/108 [==============================] - 0s 831us/sample - loss: 0.0379 - mae: 0.1387 - mse: 0.0379 - val_loss: 0.0359 - val_mae: 0.1578 - val_mse: 0.0359
Epoch 54/100
 90/108 [========================>.....] - ETA: 0s - loss: 0.0348 - mae: 0.1391 - mse: 0.0348
Epoch 00054: val_loss did not improve from 0.03098
108/108 [==============================] - 0s 820us/sample - loss: 0.0414 - mae: 0.1504 - mse: 0.0414 - val_loss: 0.0456 - val_mae: 0.1894 - val_mse: 0.0456
Epoch 55/100
 90/108 [========================>.....] -

Epoch 78/100
 85/108 [======================>.......] - ETA: 0s - loss: 0.0401 - mae: 0.1468 - mse: 0.0401
Epoch 00078: val_loss did not improve from 0.02972
108/108 [==============================] - 0s 846us/sample - loss: 0.0401 - mae: 0.1475 - mse: 0.0401 - val_loss: 0.0491 - val_mae: 0.1952 - val_mse: 0.0491
Epoch 79/100
 90/108 [========================>.....] - ETA: 0s - loss: 0.0382 - mae: 0.1365 - mse: 0.0382
Epoch 00079: val_loss did not improve from 0.02972
108/108 [==============================] - 0s 797us/sample - loss: 0.0371 - mae: 0.1344 - mse: 0.0371 - val_loss: 0.0362 - val_mae: 0.1649 - val_mse: 0.0362
Epoch 80/100
 90/108 [========================>.....] - ETA: 0s - loss: 0.0419 - mae: 0.1504 - mse: 0.0419
Epoch 00080: val_loss did not improve from 0.02972
108/108 [==============================] - 0s 798us/sample - loss: 0.0381 - mae: 0.1444 - mse: 0.0381 - val_loss: 0.0352 - val_mae: 0.1550 - val_mse: 0.0352
Epoch 81/100
 90/108 [========================>.....] -

Epoch 29/100
 90/108 [========================>.....] - ETA: 0s - loss: 0.0550 - mae: 0.1720 - mse: 0.0550
Epoch 00029: val_loss did not improve from 0.03794
108/108 [==============================] - 0s 809us/sample - loss: 0.0538 - mae: 0.1709 - mse: 0.0538 - val_loss: 0.0668 - val_mae: 0.2144 - val_mse: 0.0668
Epoch 30/100
 85/108 [======================>.......] - ETA: 0s - loss: 0.0524 - mae: 0.1690 - mse: 0.0524
Epoch 00030: val_loss did not improve from 0.03794
108/108 [==============================] - 0s 835us/sample - loss: 0.0476 - mae: 0.1589 - mse: 0.0476 - val_loss: 0.0639 - val_mae: 0.2099 - val_mse: 0.0639
Epoch 31/100
 85/108 [======================>.......] - ETA: 0s - loss: 0.0478 - mae: 0.1578 - mse: 0.0478
Epoch 00031: val_loss did not improve from 0.03794
108/108 [==============================] - 0s 829us/sample - loss: 0.0477 - mae: 0.1583 - mse: 0.0477 - val_loss: 0.0590 - val_mae: 0.2045 - val_mse: 0.0590
Epoch 32/100
 90/108 [========================>.....] -

...

Epoch 86/100
 80/108 [=====================>........] - ETA: 0s - loss: 0.0397 - mae: 0.1334 - mse: 0.0397
Epoch 00086: val_loss did not improve from 0.03114
108/108 [==============================] - 0s 876us/sample - loss: 0.0354 - mae: 0.1221 - mse: 0.0354 - val_loss: 0.0717 - val_mae: 0.2319 - val_mse: 0.0717
Epoch 87/100
100/108 [==========================>...] - ETA: 0s - loss: 0.0347 - mae: 0.1144 - mse: 0.0347
Epoch 00087: val_loss did not improve from 0.03114
108/108 [==============================] - 0s 755us/sample - loss: 0.0329 - mae: 0.1112 - mse: 0.0329 - val_loss: 0.0800 - val_mae: 0.2417 - val_mse: 0.0800
Epoch 88/100
 90/108 [========================>.....] - ETA: 0s - loss: 0.0310 - mae: 0.1099 - mse: 0.0310
Epoch 00088: val_loss did not improve from 0.03114
108/108 [==============================] - 0s 791us/sample - loss: 0.0332 - mae: 0.1133 - mse: 0.0332 - val_loss: 0.0654 - val_mae: 0.2235 - val_mse: 0.0654
Epoch 89/100
 95/108 [=========================>....] -

Epoch 11/100
 90/108 [========================>.....] - ETA: 0s - loss: 0.0324 - mae: 0.1093 - mse: 0.0324
Epoch 00011: val_loss did not improve from 0.24971
108/108 [==============================] - 0s 799us/sample - loss: 0.0323 - mae: 0.1132 - mse: 0.0323 - val_loss: 0.3165 - val_mae: 0.5295 - val_mse: 0.3165
Epoch 12/100
 90/108 [========================>.....] - ETA: 0s - loss: 0.0360 - mae: 0.1144 - mse: 0.0360
Epoch 00012: val_loss did not improve from 0.24971
108/108 [==============================] - 0s 800us/sample - loss: 0.0318 - mae: 0.1070 - mse: 0.0318 - val_loss: 0.3010 - val_mae: 0.5150 - val_mse: 0.3010
Epoch 13/100
 95/108 [=========================>....] - ETA: 0s - loss: 0.0300 - mae: 0.0987 - mse: 0.0300
Epoch 00013: val_loss improved from 0.24971 to 0.23956, saving model to model_checkpoint_w_val.h5
108/108 [==============================] - 0s 970us/sample - loss: 0.0295 - mae: 0.0988 - mse: 0.0295 - val_loss: 0.2396 - val_mae: 0.4509 - val_mse: 0.2396
Epoch 14

108/108 [==============================] - 0s 972us/sample - loss: 0.0202 - mae: 0.0832 - mse: 0.0202 - val_loss: 0.1095 - val_mae: 0.2987 - val_mse: 0.1095
Epoch 37/100
100/108 [==========================>...] - ETA: 0s - loss: 0.0202 - mae: 0.0846 - mse: 0.0202
Epoch 00037: val_loss did not improve from 0.10947
108/108 [==============================] - 0s 735us/sample - loss: 0.0194 - mae: 0.0839 - mse: 0.0194 - val_loss: 0.1797 - val_mae: 0.3935 - val_mse: 0.1797
Epoch 38/100
 90/108 [========================>.....] - ETA: 0s - loss: 0.0179 - mae: 0.0824 - mse: 0.0179
Epoch 00038: val_loss improved from 0.10947 to 0.10718, saving model to model_checkpoint_w_val.h5
108/108 [==============================] - 0s 978us/sample - loss: 0.0201 - mae: 0.0902 - mse: 0.0201 - val_loss: 0.1072 - val_mae: 0.2950 - val_mse: 0.1072
Epoch 39/100
100/108 [==========================>...] - ETA: 0s - loss: 0.0209 - mae: 0.0834 - mse: 0.0209
Epoch 00039: val_loss did not improve from 0.10718
108/108 

Epoch 62/100
 95/108 [=========================>....] - ETA: 0s - loss: 0.0199 - mae: 0.0889 - mse: 0.0199
Epoch 00062: val_loss did not improve from 0.09304
108/108 [==============================] - 0s 787us/sample - loss: 0.0188 - mae: 0.0873 - mse: 0.0188 - val_loss: 0.1058 - val_mae: 0.2810 - val_mse: 0.1058
Epoch 63/100
 95/108 [=========================>....] - ETA: 0s - loss: 0.0133 - mae: 0.0757 - mse: 0.0133
Epoch 00063: val_loss did not improve from 0.09304
108/108 [==============================] - 0s 786us/sample - loss: 0.0156 - mae: 0.0803 - mse: 0.0156 - val_loss: 0.1285 - val_mae: 0.3203 - val_mse: 0.1285
Epoch 64/100
 90/108 [========================>.....] - ETA: 0s - loss: 0.0165 - mae: 0.0777 - mse: 0.0165
Epoch 00064: val_loss improved from 0.09304 to 0.09155, saving model to model_checkpoint_w_val.h5
108/108 [==============================] - 0s 972us/sample - loss: 0.0152 - mae: 0.0761 - mse: 0.0152 - val_loss: 0.0915 - val_mae: 0.2578 - val_mse: 0.0915
Epoch 65

Epoch 88/100
 90/108 [========================>.....] - ETA: 0s - loss: 0.0137 - mae: 0.0693 - mse: 0.0137
Epoch 00088: val_loss did not improve from 0.07773
108/108 [==============================] - 0s 792us/sample - loss: 0.0125 - mae: 0.0664 - mse: 0.0125 - val_loss: 0.1224 - val_mae: 0.2862 - val_mse: 0.1224
Epoch 89/100
 95/108 [=========================>....] - ETA: 0s - loss: 0.0150 - mae: 0.0784 - mse: 0.0150
Epoch 00089: val_loss did not improve from 0.07773
108/108 [==============================] - 0s 788us/sample - loss: 0.0138 - mae: 0.0764 - mse: 0.0138 - val_loss: 0.3389 - val_mae: 0.5381 - val_mse: 0.3389
Epoch 90/100
 85/108 [======================>.......] - ETA: 0s - loss: 0.0187 - mae: 0.0943 - mse: 0.0187
Epoch 00090: val_loss did not improve from 0.07773
108/108 [==============================] - 0s 850us/sample - loss: 0.0164 - mae: 0.0878 - mse: 0.0164 - val_loss: 0.1742 - val_mae: 0.3699 - val_mse: 0.1742
Epoch 91/100
 85/108 [======================>.......] -